In [1]:
from utilities import *
from core.initial import Insertions
from docplex.cp.model import *
from IPython.display import display


In [2]:
datafile = "C:/Users/aznavouridis.k/My Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"

d = DataProvider(filepath=datafile, route='910')

model = CSPModel(d)
model.build_model()

model.data.head()

,initial_depot,final_depot,relief_point,time,trip_duration,start_time,end_time
trip,,,,,,,
0,Fix Station,Fix Station,NaN,05:00,40,300,340
1,Fix Station,Fix Station,NaN,05:20,40,320,360
2,Fix Station,Fix Station,NaN,05:55,40,355,395
3,Fix Station,Fix Station,NaN,06:20,44,380,424
4,Fix Station,Fix Station,NaN,06:40,44,400,444


In [ ]:
initial = Insertions(model)
initial.solve()

In [3]:
trips = [interval_var(start=(trip.start_time, trip.start_time),
                      end=(trip.end_time, trip.end_time),
                      size=trip.duration,
                      name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


In [4]:
ntrips = len(model.trips)
nduties = len(model.trips)

sub = CpoModel(name="Pricing_Subproblem")

# Variables

min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=model.constraints.shift_span,
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(nduties)]

trip2trip = integer_var_list(size=ntrips,
                             min=0,
                             max=ntrips + 1,
                             name='Trip2Trip')

trip2duty = integer_var_list(size=ntrips,
                             min=0,
                             max=nduties,
                             name='Trip2Duty')

start_times = [[integer_var(min=0,
                            max=model.constraints.shift_span,
                            name=f"StartTime-{i}-{j}")
               for j in range(nduties)] for i in range(ntrips)]

cdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.continuous_driving,
                       name="CDT")

tdt = integer_var_list(size=nduties,
                       min=0,
                       max=model.constraints.total_driving,
                       name="TDT")

# Constraints

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_times[i] <= model.start_times[j]))

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(
            trip2trip[i] == j, model.end_locs[i] <= model.start_locs[j]))

for i in range(ntrips):
    sub.add(trip2trip[i] != i)

for i in range(ntrips):
    for j in range(ntrips):
        sub.add(sub.if_then(trip2trip[i] == j, trip2duty[i] == trip2duty[j]))


In [ ]:
def report_solution(cpsol: CpoSolveResult):
    trips_per_duty = {}
    for i in range(ntrips):
        _out = f"{i:>2} -> {cpsol[trip2trip[i]]} | Duty: {cpsol[trip2duty[i]]}"
        # print(_out)

        duty_id = cpsol[trip2duty[i]]

        if duty_id in trips_per_duty:
            trips_per_duty[duty_id].append(i)
        else:
            trips_per_duty[duty_id] = []
            trips_per_duty[duty_id].append(i)

    print(f'\n\nTotal Duties: {len(trips_per_duty.keys())}')

    for duty_id, duty_trips in trips_per_duty.items():
        df_trips = model.data.loc[duty_trips]

        span = df_trips[end_time].max() - df_trips[start_time].min()

        print(
            f'\n\n>>> Duty {duty_id} - Trips: {len(duty_trips)} - Drive Time: {df_trips[trip_duration].sum()} - Shift Span: {span}\n')
        display(df_trips)

In [18]:
NTRIPS = len(model.trips)
NDUTIES = 10

sub = CpoModel(name="Pricing_Subproblem")

min_start = model.data[start_time].min()
max_start = model.data[start_time].max()
min_end = model.data[end_time].min()
max_end = model.data[end_time].max()

# trips = [interval_var(start=(trip.start_time, trip.start_time),
#                       end=(trip.end_time, trip.end_time),
#                       size=trip.duration,
#                       name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]


duties = [interval_var(start=(min_start, max_start),
                       end=(min_end, max_end),
                       size=(0, model.constraints.shift_span),
                       name=f"Duty_{i}",
                       optional=True)
          for i in range(NDUTIES)]

breaks = [interval_var(size=model.constraints.break_time,
                       name=f"BreakTime_{i}",
                       optional=True)
          for i in range(NDUTIES)]

trip2duty = {}
for t, trip in enumerate(model.trips):
    for d in range(NDUTIES):
        trip2duty[(t, d)] = sub.interval_var(start=(trip.start_time, trip.start_time),
                                             end=(trip.end_time, trip.end_time),
                                             size=trip.duration,
                                             name=f"Duty_{d:02}-Trip_{t:02}",
                                             optional=True)

In [19]:
for d in range(NDUTIES):
    sub.add(sub.span(duties[d], [trip2duty[(t, d)] for t in range(NTRIPS)]))

for d in range(NDUTIES):
    sub.add(sub.no_overlap([trip2duty[(t, d)] for t in range(NTRIPS)]))

for t in range(NTRIPS):
    sub.add(sub.sum([sub.presence_of(trip2duty[(t, d)]) for d in range(NDUTIES)]) == 1)

In [ ]:
cdt = {}
tdt = {}

for d in range(nduties):
    cdt[d] = sub.step_at(0, 0)
    tdt[d] = sub.step_at(0, 0)
    for t in range(ntrips):
        cdt[d] += sub.pulse(trip2duty[(t,d)], model.durations[t])
        tdt[d] += sub.pulse(trip2duty[(t,d)], model.durations[t])

    sub.add(cdt[d] <= model.constraints.continuous_driving)
    sub.add(tdt[d] <= model.constraints.total_driving)
  

In [20]:
obj = sub.sum([sub.presence_of(duty) for duty in duties])
sub.add(sub.minimize(obj))

In [21]:
msol = sub.solve()

In [23]:
msol.get_infos()

{'DepthFirstIdleTime': 0,
 'EffectiveDepthFirstWorkers': 0,
 'EffectiveIterativeDivingWorkers': 1,
 'EffectiveMultiPointWorkers': 0,
 'EffectiveOptimalityTolerance': 'infinity',
 'EffectiveRestartWorkers': 3,
 'EffectiveWorkers': 4,
 'ExtractionTime': 0.013,
 'FailStatus': 'SearchHasFailedNormally',
 'IterativeDivingIdleTime': 0,
 'MemoryUsage': 16154093,
 'MultiPointIdleTime': 0,
 'NumberOfAuxiliaryVariables': 0,
 'NumberOfBranches': 19803221,
 'NumberOfChoicePoints': 542308,
 'NumberOfConstraints': 78,
 'NumberOfConstraintsAdded': 0,
 'NumberOfConstraintsAggregated': 0,
 'NumberOfConstraintsGenerated': 0,
 'NumberOfConstraintsRemoved': 0,
 'NumberOfCriteria': 1,
 'NumberOfEngineConstraints': 83,
 'NumberOfEngineVariables': 602,
 'NumberOfErrors': 0,
 'NumberOfFails': 528051,
 'NumberOfIntegerVariables': 0,
 'NumberOfIntervalVariables': 590,
 'NumberOfModelVariables': 600,
 'NumberOfPresolveTransformations': 0,
 'NumberOfSequenceVariables': 10,
 'NumberOfSolutions': 2,
 'NumberOfState

In [25]:
msol.get_objective_bounds()

(7,)

In [26]:
msol.get_objective_gaps()

(0,)

In [28]:
msol.get_parameters()

{'AutomaticReplay': 'On',
 'BranchLimit': 'intmax',
 'CPOFileStrictIdCount': 'Off',
 'ChoicePointLimit': 'intmax',
 'ConflictDefinition': 'ConflictInfeasible',
 'ConflictRefinerAlgorithm': 3,
 'ConflictRefinerBranchLimit': 'intmax',
 'ConflictRefinerFailLimit': 'intmax',
 'ConflictRefinerIterationLimit': 'intmax',
 'ConflictRefinerOnLabeledConstraints': 'On',
 'ConflictRefinerOnVariables': 'Off',
 'ConflictRefinerTimeLimit': 'infinity',
 'ConflictRefinerWriteMode': 1,
 'DepthFirstWorkers': 0,
 'EliminatePresolvedModel': 'On',
 'FailLimit': 'intmax',
 'ForbidIncludeDirective': 'Off',
 'IterativeDivingWorkers': 0,
 'KPIDisplay': 'SingleLine',
 'LogDisplayWorkerIdleTime': 'Off',
 'MemoryDisplay': 'On',
 'MultiPointWorkers': 0,
 'ObjectiveLimit': 'infinity',
 'ParallelCommunicateEachSolution': 'Off',
 'ParallelEventQueueCapacity': 'Auto',
 'ParallelIterativeDivingWorkerIdMask': 0,
 'ParallelMode': 'Auto',
 'ParallelOptimizeSingleWorker': 'On',
 'ParallelReduceMemoryFragmentation': 'Off',
 

In [30]:
msol.get_search_status()

'SearchCompleted'

In [33]:
msol.get_solution()

In [34]:
msol.get_solve_status()

'Optimal'

In [40]:
msol.get_value(trip2duty[8, 6])

(465, 513, 48)

In [45]:
msol.print_solution()

-------------------------------------------------------------------------------
Model constraints: 78, variables: integer: 0, interval: 590, sequence: 10
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 58.19 sec
-------------------------------------------------------------------------------
Objective values: (7,), bounds: (7,), gaps: (0,)
Duty_0: (start=710, end=1284, size=574, length=574)
Duty_1: absent
Duty_2: (start=320, end=854, size=534, length=534)
Duty_3: (start=895, end=1460, size=565, length=565)
Duty_4: (start=355, end=819, size=464, length=464)
Duty_5: (start=845, end=1425, size=580, length=580)
Duty_6: (start=465, end=1062, size=597, length=597)
Duty_7: absent
Duty_8: absent
Duty_9: (start=300, end=769, size=469, length=469)
Duty_00-Trip_00: absent
Duty_00-Trip_01: absent
Duty_00-Trip_02: absent
Duty_00-Trip_03: absent
Duty_00-Trip_04: absent
Duty_00-Trip_05: absent
Duty_00-Trip_06: absent
Duty_00-Trip_07: absent
Duty_00

In [47]:
msol.process_infos

{'ModelBuildTime': 6.700611114501953,
 'TotalUtf8DecodeTime': 0.003996610641479492,
 'TotalDataReceiveSize': 141135,
 'ProxyVersion': 10,
 'AngelVersion': 10,
 'SourceDate': 'Nov 12 2020',
 'SolverVersion': '20.1.0.0',
 'SolverAgent': 'local',
 'ModelCompileTime': 0.016951322555541992,
 'ModelDataSize': 73970,
 'TotalUtf8EncodeTime': 0.0,
 'TotalDataSendTime': 0.0007092952728271484,
 'TotalDataSendSize': 74004,
 'ModelSubmitTime': 0.021655797958374023,
 'TotalLogDataSize': 87180,
 'TotalJsonParseTime': 0.0,
 'JsonFormatVersion': '1.17',
 'TotalSolveTime': 58.18756413459778}

In [49]:
msol.write("C:/Users/aznavouridis.k/Desktop/test.txt")

In [64]:

for d in range(NDUTIES):
    if msol[duties[d]]:
        print(f"\n> Duty {d} : {msol[duties[d]]}")
        for t in range(NTRIPS):
            if msol[trip2duty[(t, d)]]:
                print(f"  - Trip {t} : {msol[trip2duty[(t, d)]]}")




> Duty 0 : (710, 1284, 574)
  - Trip 23 : (710, 754, 44)
  - Trip 28 : (795, 839, 44)
  - Trip 32 : (860, 908, 48)
  - Trip 36 : (925, 973, 48)
  - Trip 39 : (975, 1027, 52)
  - Trip 43 : (1045, 1097, 52)
  - Trip 46 : (1105, 1149, 44)
  - Trip 48 : (1150, 1194, 44)
  - Trip 52 : (1240, 1284, 44)

> Duty 2 : (320, 854, 534)
  - Trip 1 : (320, 360, 40)
  - Trip 5 : (420, 468, 48)
  - Trip 9 : (480, 532, 52)
  - Trip 13 : (545, 597, 52)
  - Trip 17 : (610, 654, 44)
  - Trip 21 : (675, 719, 44)
  - Trip 25 : (745, 789, 44)
  - Trip 29 : (810, 854, 44)

> Duty 3 : (895, 1460, 565)
  - Trip 34 : (895, 943, 48)
  - Trip 40 : (995, 1047, 52)
  - Trip 44 : (1060, 1112, 52)
  - Trip 47 : (1130, 1174, 44)
  - Trip 50 : (1190, 1234, 44)
  - Trip 54 : (1305, 1345, 40)
  - Trip 57 : (1420, 1460, 40)

> Duty 4 : (355, 819, 464)
  - Trip 2 : (355, 395, 40)
  - Trip 4 : (400, 444, 44)
  - Trip 7 : (450, 498, 48)
  - Trip 11 : (515, 567, 52)
  - Trip 15 : (580, 632, 52)
  - Trip 19 : (645, 689, 44)
  

In [ ]:
msol = sub.solve()

report_solution(msol)

In [ ]:
sub.end_of(trips[i]) <= sub.end_of(trips[j]))

In [ ]:
start_of(trips[1]) <= 200

In [ ]:
less_or_equal(start_of(trips[0]), start_of(trips[1]))

In [ ]:
for i in range(len(trips)):
    for j in range(1, len(trips)):
        print(less_or_equal(trips[i].get_start() , trips[j].get_start()))


In [ ]:
for i in range(ntrips):
    sub.add(trip2trip[i] != 0)
    # sub.add(sum([trip2trip[j] == i for j in range(ntrips+1)]) == 1)

In [ ]:
for i in range(ntrips-1):
    for j in range(i + 1, ntrips):
        sub.add(end_before_start(trips[i], trips[j]))

In [ ]:
has_break = [[binary_var(name=f'Break_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]
has_rest = [[binary_var(name=f'Rest_{i}_{j}') for j in range(nduties)] for i in range(ntrips)]

In [ ]:
trip_seq = sequence_var(trips)

In [ ]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

In [17]:
a = 11
print(f"{a:02}")

11
